In [9]:
import os

from collections import defaultdict, Counter
import numpy as np

In [3]:
def Input(day):
    # Read an input
    filename = 'data/{}.txt'.format(day)
    return open(filename)

def Lines(day):
    # Read the input and convert to an array of strings.
    return Input(day).read().splitlines()

def vector(day):
    # Read the input, and convert it to a vector of numbers
    return list(map(float, Lines(day)))

## Day 1

In [4]:
# Read input
inp = vector(1)
           
# The answer to the first one is just the sum
print(sum(inp))

533.0


In [5]:
# For the second part, use a set to determine if we've seen 
# a frequency before
v = set()
ii = 0
s = 0
while not found:
    s = s + inp[ii % len(inp)]
    if s in v:
        print("Found freq: {} at iteration {} ({} passes)".format(
                s, ii, ii % len(inp)))
        break
    v.add(s)
    ii = ii + 1

NameError: name 'found' is not defined

## Day 2

In [28]:
def checksum(s):
    cnt = Counter()
    for l in s:
        cnt[l] += 1
    return (2 in cnt.values(), 3 in cnt.values())

assert(checksum('abcdef') == (False, False))
assert(checksum('bababc') == (True, True))
assert(checksum('abbcde') == (True, False))
assert(checksum('abcccd') == (False, True))
assert(checksum('aabcdd') == (True, False))
assert(checksum('abcdee') == (True, False))
assert(checksum('ababab') == (False, True))

inp = Lines(2)

twos = 0
threes = 0

for line in inp:
    cnt = checksum(line)
    if cnt[0]:
        twos += 1
    if cnt[1]:
        threes += 1
        
print('Twos: {}, Threes: {}, Checksum: {}'.format(twos, threes, twos * threes))     

Twos: 245, Threes: 36, Checksum: 8820


In [44]:
def diff(s1, s2):
    count = 0
    for (c1, c2) in zip(s1, s2):
        if c1 != c2:
            count += 1
    return count
    
fline1 = None
fline2 = None
    
for line1 in inp:
    for line2 in inp:
        if diff(line1, line2) == 1:
            fline1 = line1
            fline2 = line2
            break
    if fline1:
        break
        
print(fline1)
print(fline2)

out = ''
for (c1, c2) in zip(fline1, fline2):
    if c1 == c2:
        out += c1
print(out)

bpacnmglhizqygfsjixtlkwudr
bpacnmglhizqygfsjixtukwudr
bpacnmglhizqygfsjixtkwudr
